<a href="https://colab.research.google.com/github/subin6985/20242R0136COSE47402/blob/main/FinalProject/Deep_Learning_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install transformers
!pip install evaluate
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=ro

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Trainer, TrainingArguments
from evaluate import load

In [3]:
dataset = load_dataset("cnn_dailymail", "3.0.0")
print(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})


In [4]:
# 8:1:1 비율로 데이터셋 분할
train_test_split = dataset["train"].train_test_split(test_size=0.2, seed=42)
val_test_split = train_test_split["test"].train_test_split(test_size=0.5, seed=42)
train_dataset = train_test_split["train"]
val_dataset = val_test_split["train"]
test_dataset = val_test_split["test"]

# 데이터 분할 상태 확인
print(f"Train: {len(train_dataset)}, Validation: {len(val_dataset)}, Test: {len(test_dataset)}")

Train: 229690, Validation: 28711, Test: 28712


In [5]:
# 모델과 토크나이저 준비
model_name = "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [6]:
# 전처리 함수 정의
def preprocess_function(examples):
    # 입력 데이터: "Summarize:" 프롬프트 추가
    inputs = ["Summarize: " + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # 라벨 데이터 처리
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=128, truncation=True, padding="max_length")

    # 라벨의 패딩 토큰을 -100으로 변경
    labels["input_ids"] = [
        [(label if label != tokenizer.pad_token_id else -100) for label in label_list]
        for label_list in labels["input_ids"]
    ]
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [7]:
# 샘플링 데이터셋 추출
train_dataset_sampled = train_dataset.select(range(500))
val_dataset_sampled = val_dataset.select(range(100))
test_dataset_sampled = test_dataset.select(range(100))

# 샘플링 데이터셋 전처리
train_dataset = train_dataset_sampled.map(preprocess_function, batched=True)
val_dataset = val_dataset_sampled.map(preprocess_function, batched=True)
test_dataset = test_dataset_sampled.map(preprocess_function, batched=True)


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [8]:
# 훈련 파라미터 설정
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01,
    save_strategy="epoch",
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=100,
    fp16=True,
    report_to="none"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True)

# Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# 모델 학습
trainer.train()

<ipython-input-9-22a562d6e961>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,2.728600,2.254650
2,2.181400,2.218722


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=250, training_loss=2.3572749938964845, metrics={'train_runtime': 6099.5343, 'train_samples_per_second': 0.164, 'train_steps_per_second': 0.041, 'total_flos': 304868229120000.0, 'train_loss': 2.3572749938964845, 'epoch': 2.0})

In [10]:
# 평가
metrics = trainer.evaluate(test_dataset)
print(metrics)

{'eval_loss': 2.1323797702789307, 'eval_runtime': 160.3784, 'eval_samples_per_second': 0.624, 'eval_steps_per_second': 0.156, 'epoch': 2.0}


In [11]:
# 테스트 데이터셋으로 요약 생성
def generate_summary(examples):
    inputs = ["Summarize: " + doc for doc in examples["article"]]
    inputs = tokenizer(inputs, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = model.generate(inputs["input_ids"], max_length=128, num_beams=4, early_stopping=True)
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

In [20]:
# 테스트 데이터 일부에 대해 요약 생성
test_sample = test_dataset.select(range(10))
generated_summaries = generate_summary(test_sample)

rouge = load("rouge")

# ROUGE 점수 계산
results = rouge.compute(
    predictions=generated_summaries,
    references=test_sample["highlights"]
)
print(results)

{'rouge1': 0.40532013330143024, 'rouge2': 0.19236403719694156, 'rougeL': 0.28694463827374717, 'rougeLsum': 0.38635563699268394}


In [21]:
# 요약 확인
for i, summary in enumerate(generated_summaries[:5]):  # 처음 5개만 슬라이싱
    print(f"Original: {test_sample[i]['highlights']}")
    print(f"Generated: {summary}")
    print("-" * 80)

Original: Aung San Suu Kyi is released Saturday .
She has been under house arrest for much of the past two decades .
She has defiantly challenged the authority of the military junta .
She likens Myanmar's plight to South African apartheid .
Generated: Aung San Suu Kyi is the very embodiment of Myanmar's long struggle for democracy .
The 65-year-old human rights activist has endured house arrest for much of the past two decades and, perhaps, has become the world's most recognizable political prisoner .
--------------------------------------------------------------------------------
Original: An energized conservative electorate helps Republicans to historic gain in midterms .
Republicans nab at least 60 more House seats, based on CNN analysis of exit poll data .
Senate Majority Leader Harry Reid defeats Republican Sharron Angle in Nevada .
President Obama calls House Minority Leader John Boehner to congratulate him .
Generated: President Barack Obama called House Minority Leader John Bo

In [5]:
# Pegasus
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

model_name = "google/pegasus-xsum"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

In [6]:
# 데이터 전처리
def preprocess_function(examples):
    inputs = ["Summarize: " + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = [
        [(label if label != tokenizer.pad_token_id else -100) for label in label_list]
        for label_list in labels["input_ids"]
    ]
    return model_inputs

train_dataset_sampled = train_dataset.select(range(500))
val_dataset_sampled = val_dataset.select(range(100))
test_dataset_sampled = test_dataset.select(range(100))

train_dataset = train_dataset_sampled.map(preprocess_function, batched=True, remove_columns=["article", "highlights", "id"])
val_dataset = val_dataset_sampled.map(preprocess_function, batched=True, remove_columns=["article", "highlights", "id"])
test_dataset = test_dataset_sampled.map(preprocess_function, batched=True, remove_columns=["article", "highlights", "id"])

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [7]:
# 모델 학습
training_args = TrainingArguments(
    output_dir="./results_pegasus",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01,
    save_strategy="epoch",
    save_total_limit=2,
    logging_dir="./logs_pegasus",
    logging_steps=100,
    fp16=True,
    report_to="none",
    remove_unused_columns=False
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [8]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

<ipython-input-8-a361b4c511e1>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,2.403600,1.969363
2,2.085300,1.946880


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 64, 'num_beams': 8, 'length_penalty': 0.6}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=250, training_loss=2.180014343261719, metrics={'train_runtime': 446.3743, 'train_samples_per_second': 2.24, 'train_steps_per_second': 0.56, 'total_flos': 1444732207104000.0, 'train_loss': 2.180014343261719, 'epoch': 2.0})

In [10]:
# Pegasus 평가
metrics = trainer.evaluate(test_dataset)
print(metrics)

{'eval_loss': 1.910831332206726, 'eval_runtime': 4.1103, 'eval_samples_per_second': 24.329, 'eval_steps_per_second': 6.082, 'epoch': 2.0}


In [16]:
import torch

# 테스트 데이터셋으로 요약 생성
def generate_summary(examples):
    inputs = torch.tensor(examples["input_ids"]).to(model.device)
    attention_mask = torch.tensor(examples["attention_mask"]).to(model.device)
    outputs = model.generate(inputs, attention_mask=attention_mask, max_length=128, num_beams=4, early_stopping=True)
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

# 테스트 데이터 일부에 대해 요약 생성
test_sample = test_dataset.select(range(10))
generated_summaries = generate_summary(test_sample)

In [17]:
rouge = load("rouge")

# test_sample["labels"]에서 -100을 제외하고 디코딩
decoded_references = [
    tokenizer.decode([token_id for token_id in label if token_id != -100], skip_special_tokens=True)
    for label in test_sample["labels"]
]

# ROUGE 점수 계산
results = rouge.compute(
    predictions=generated_summaries,
    references=decoded_references
)
print(results)

{'rouge1': 0.3142400250577112, 'rouge2': 0.1416836949481469, 'rougeL': 0.21456648522948274, 'rougeLsum': 0.2151229315008123}
